In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root is set:", project_root)


Project root is set: /home/homeflo/workspace/project_trading/Trading-Bot


#### === Step 1: Load data  ===

In [3]:
# symbol =  "GBP_USD_M1_09-01-2025_10-01-2025_with_volume.pkl" #"GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl"  #"XAUUSD_H1.csv"
# symbol =  "GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl"
# symbol =  "GBP_USD_M1_10-27-2025_10-28-2025_with_volume.pkl"
# symbol = "USD_JPY_M1_10-27-2025_10-28-2025_with_volume.pkl"
symbol = "GBP_USD_M1_09-18-2025_09-25-2025_with_volume.pkl"
print(f"📊 Loading data for {symbol} with days_back=60...")
ltf1 = "1M"
ltf2 = "2M"
mtf = "15M"
htf = "4H"
# Use the same data loading function
from core.data_loader import load_and_resample
resampled = load_and_resample(f"../data/{symbol}", days_back=60)
ltf1_data = resampled.get(ltf1)
ltf2_data = resampled.get(ltf2)
mtf_data = resampled.get(mtf)
htf_data = resampled.get(htf)

if ltf1_data is None or ltf1_data.empty:
    print(f"❌ ERROR: No data loaded for the '{ltf1}' timeframe.")
elif ltf2_data is None or ltf2_data.empty:
    print(f"❌ ERROR: No data loaded for the '{ltf2}' timeframe.")
elif mtf_data is None or mtf_data.empty:
    print(f"❌ ERROR: No data loaded for the '{mtf}' timeframe.")
elif htf_data is None or htf_data.empty:
    print(f"❌ ERROR: No data loaded for the '{htf}' timeframe.")
    
else:
    print(f"📊 Loaded {len(ltf1_data):5} '{ltf1}' candles from \t{ltf1_data.index.min()} to {ltf1_data.index.max()}")
    print(f"📊 Loaded {len(ltf2_data):5} '{ltf2}' candles from \t{ltf2_data.index.min()} to {ltf2_data.index.max()}")
    print(f"📊 Loaded {len(mtf_data):5} '{mtf}' candles from \t{mtf_data.index.min()} to {mtf_data.index.max()}")
    print(f"📊 Loaded {len(htf_data):5} '{htf}' candles from \t{htf_data.index.min()} to {htf_data.index.max()}")
    data = {ltf1: ltf1_data, ltf2: ltf2_data, mtf: mtf_data, htf: htf_data}

📊 Loading data for GBP_USD_M1_09-18-2025_09-25-2025_with_volume.pkl with days_back=60...
📊 Loaded  7171 '1M' candles from 	2025-09-18 00:00:00+00:00 to 2025-09-25 00:00:00+00:00
📊 Loaded  3581 '2M' candles from 	2025-09-18 00:00:00+00:00 to 2025-09-24 23:58:00+00:00
📊 Loaded   470 '15M' candles from 	2025-09-18 00:00:00+00:00 to 2025-09-24 23:45:00+00:00
📊 Loaded    25 '4H' candles from 	2025-09-18 00:00:00+00:00 to 2025-09-24 16:00:00+00:00


#### === Step 2: Analyse market structure ===

In [4]:
print("🔍 Analyzing market structure...")
from core.structure_builder import analyse_market_structure
from time import perf_counter_ns

t = perf_counter_ns()
swing_high_low_test, trend_test = analyse_market_structure(mtf_data, based_on_atr=True, prominence_factor=1)#dropna()
swing_high_low_test = swing_high_low_test.dropna()
print(f"⏱️  Market structure analysis with ATR took \t {(perf_counter_ns() - t)/ 1_000_000} ms")

t = perf_counter_ns()
swing_high_low_test_2, trend_test_2 = analyse_market_structure(mtf_data, based_on_atr=False)#.dropna()
swing_high_low_test_2 = swing_high_low_test_2.dropna()

print(f"⏱️  Market structure analysis without ATR took \t {(perf_counter_ns() - t) / 1_000_000} ms")

🔍 Analyzing market structure...
⏱️  Market structure analysis with ATR took 	 15.901852 ms
⏱️  Market structure analysis without ATR took 	 11.848717 ms


In [179]:
print("🔍 Analyzing market structure...")
from core.structure_builder import analyse_market_structure

prominence_factor ={
    ltf1: 1,
    ltf2: 1,
    mtf: 1,
    htf: 0.5
}

structure = {}
trends = {}
for tf, df in data.items():
    analysis, trend = analyse_market_structure(df, swing_length=10, trend_window=4, prominence_factor=prominence_factor[tf])
    structure[tf] = analysis.dropna()
    trends[tf] = trend
    print(f"📊 {tf:3}: Total structure points: {len(structure[tf].dropna()):5}, Trend: {trend}")

🔍 Analyzing market structure...
📊 1M : Total structure points:   682, Trend: uptrend
📊 2M : Total structure points:   358, Trend: uptrend
📊 15M: Total structure points:    34, Trend: downtrend
📊 4H : Total structure points:     4, Trend: downtrend


In [106]:
from importlib import reload
from core import structure_builder
reload(structure_builder)

<module 'core.structure_builder' from '/home/homeflo/workspace/project_trading/Trading-Bot/core/structure_builder.py'>

In [108]:
structure_mtf = structure[mtf]


#### === Step 3: Detect market events ===

In [188]:
print("🎯 Detecting market events...")
from core.smart_money_concepts import MarketStructureAnalyzer
analyzer = MarketStructureAnalyzer(confidence_threshold=0.5)

all_events = {}
extern_structure =  {}
for tf, df in data.items():
    print(f"🔍 Analyzing {tf} timeframe...")
    all_events[tf], extern_structure[tf], _ = analyzer.analyse_market_events(structure[tf], data[tf])
    print(f"📊 {tf:3}: Detected events: {len(all_events[tf]):5}\n")
    # all_events[tf] = events
    # if tf == mtf:
    #     mtf_extern_events = extern_events

mtf_events = all_events[mtf]

🎯 Detecting market events...
🔍 Analyzing 1M timeframe...
Index 0: SwingType.HL @ 1.36214 - Trend before: sideways
Index 1: SwingType.HH @ 1.36292 - Trend before: sideways
Index 2: SwingType.HL @ 1.36233 - Trend before: sideways
Index 3: SwingType.HH @ 1.36308 - Trend before: uptrend
Index 4: SwingType.LL @ 1.36122 - Trend before: uptrend
Potential Bearish CHOCH at Index 4: LL @ 1.36122 breaking HL @ 1.36233
QML: StructurePoint(timestamp=Timestamp('2025-09-18 01:00:00+0000', tz='UTC'), price=1.36308, swing_type=<SwingType.HH: 'HH'>, structure_type=<StructureType.RESISTANCE: 'Resistance'>, strong=True, retracement=127.1, retracement_structure=127.1)
CHOCH Detected at Index 4: LL @ 1.36122
Index 5: SwingType.LH @ 1.36193 - Trend before: downtrend
Index 6: SwingType.LL @ 1.36116 - Trend before: downtrend
Index 7: SwingType.LH @ 1.36172 - Trend before: downtrend
Index 8: SwingType.LL @ 1.36104 - Trend before: downtrend
Index 9: SwingType.HH @ 1.36283 - Trend before: downtrend
Potential Bull

In [187]:
from importlib import reload
from core import smart_money_concepts
reload(smart_money_concepts)

<module 'core.smart_money_concepts' from '/home/homeflo/workspace/project_trading/Trading-Bot/core/smart_money_concepts.py'>

In [315]:
from enum import Enum
from dataclasses import dataclass
from typing import Optional
import pandas as pd

class SwingType(Enum):
    HH = "HH"
    HL = "HL"
    LH = "LH"
    LL = "LL"

@dataclass
class StructurePoint:
    timestamp: pd.Timestamp
    price: float
    swing_type: SwingType
    retracement: Optional[float] = None  # Optional retracement value

swing = StructurePoint(
    timestamp=pd.Timestamp("2024-01-01 00:00:00"),
    price=1.2345,
    swing_type=SwingType.HH,
    retracement=0.618)

swing.swing_type == SwingType.HH


True

#### === Step 4: Initialize and run backtrader ===

#### === Step 5: Run backtest ===

#### === Step 6: Display results ===

#### === Step 7: Generate visualizations ===

In [9]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(mtf_data, swing_high_low_test_2.dropna(), symbol=symbol[:7], timeframe=mtf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [191]:
from importlib import reload
from utils import structure_trend_plotter
reload(structure_trend_plotter)

<module 'utils.structure_trend_plotter' from '/home/homeflo/workspace/project_trading/Trading-Bot/utils/structure_trend_plotter.py'>

In [ ]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(mtf_data, extern_structure[mtf], all_events[mtf], symbol=symbol[:7], timeframe=mtf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [11]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(htf_data, structure[htf].dropna(), symbol=symbol[:7], timeframe=htf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [12]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(ltf1_data, structure[ltf1].dropna(), symbol=symbol[:7], timeframe=ltf1)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [13]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(ltf2_data, structure[ltf2].dropna(), symbol=symbol[:7], timeframe=ltf2)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [29]:
from importlib import reload
from utils import structure_trend_plotter
reload(structure_trend_plotter)

<module 'utils.structure_trend_plotter' from '/home/homeflo/workspace/project_trading/Trading-Bot/utils/structure_trend_plotter.py'>